# Netmiko & Diffing

In [ ]:
HOST = '192.75.232.222'
PORT_NC = 830
PORT_SSH = 22
USER = 'cisco'
PASS = 'cisco'
PLATFORM = 'cisco_xr'

## Connect both netmiko and ncclient

In [ ]:
from netmiko import ConnectHandler
from ncclient import manager
from lxml import etree


def pretty_print(retval):
    print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

def get_reply(chan, eom):
    bytes = u''
    while bytes.find(eom)==-1:
        bytes += chan.recv(65535).decode('utf-8')
    return bytes

m = manager.connect(host=HOST, port=PORT_NC, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)
d = ConnectHandler(device_type=PLATFORM, ip=HOST, port=PORT_SSH, username=USER, password=PASS)

prompt = d.find_prompt()

## Sample Config To Apply

In [ ]:
config = '''interface Loopback666
description Hi
commit
end
'''

## Capture Starting Config

In [ ]:
running_config_before_text = d.send_command('show running-config')
running_config_before_xml = etree.tostring(m.get_config(source='running').data)

## Apply Config

In [ ]:
output = d.send_config_set(config.splitlines())

## Capture Post-Configlet Running Config

In [ ]:
running_config_after_text = d.send_command('show running-config')
running_config_after_xml = etree.tostring(m.get_config(source='running').data)

## XML Diff In Text Form

In [ ]:
from difflib import context_diff

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_config_before_xml.splitlines(),
                             running_config_after_xml.splitlines())))

## Rollback Using CLI

In [ ]:
rollback_output = d.send_command('rollback configuration last 1')

In [ ]:
rollback_output

## Calculate Reapply XML Document

We also exclude certain data (e.g. that from Calvados).

In [ ]:
EXCL_TAGS = [
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-man-netconf-cfg}netconf-yang',
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-crypto-sam-cfg}crypto',
    '{http://cisco.com/calvados/fpd}fpd',
    '{http://tail-f.com/ns/aaa/1.1}aaa',
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-aaa-lib-cfg}aaa',
    '{http://tail-f.com/ns/confd_dyncfg/1.0}confdConfig',
    '{http://tail-f.com/ns/mibs/SNMP-COMMUNITY-MIB/200308060000Z}SNMP-COMMUNITY-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-NOTIFICATION-MIB/200210140000Z}SNMP-NOTIFICATION-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-TARGET-MIB/200210140000Z}SNMP-TARGET-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-USER-BASED-SM-MIB/200210160000Z}SNMP-USER-BASED-SM-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-VIEW-BASED-ACM-MIB/200210160000Z}SNMP-VIEW-BASED-ACM-MIB',
    '{http://tail-f.com/ns/mibs/SNMPv2-MIB/200210160000Z}SNMPv2-MIB',
    '{http://www.cisco.com/ns/yang/Cisco-IOS-XR-sysadmin-services}service',
    '{http://www.cisco.com/panini/calvados/sdr_mgr/sdr_sm}sdr-config',
    '{http://www.cisco.com/panini/calvados/sdr_mgr/sdr_sm}private-sdr',
    '{http://openconfig.net/yang/local-routing}local-routes',
    '{http://openconfig.net/yang/interfaces}interfaces'
]
reapply = '<config>\n'
for item in etree.fromstring(running_config_after_xml).getchildren():
    if item.tag not in EXCL_TAGS:
        reapply = reapply + etree.tostring(item)
reapply = reapply + '</config>'
print(reapply)

## Reapply Netconf Config

In [ ]:
m.edit_config(reapply, format='xml', target='candidate', default_operation='merge')
m.commit()

## Get Running Config Again

In [ ]:
running_config_after_xml_apply_text = d.send_command('show running-config')
rollback_output = d.send_command('rollback configuration last 1')
print(rollback_output)

## Diff Before & After (Text Only)

In [ ]:
from difflib import context_diff

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_config_after_text.splitlines()[5:],
                             running_config_after_xml_apply_text.splitlines()[5:])))

## XML Diff In Text Form

In [ ]:
from difflib import context_diff

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_config_before_xml.splitlines(),
                             running_config_after_xml.splitlines())))

## Diff Before & After (XML)

In [ ]:
import xtdiff

EXCL_TAGS = [
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-man-netconf-cfg}netconf-yang',
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-crypto-sam-cfg}crypto',
    '{http://cisco.com/calvados/fpd}fpd',
    '{http://tail-f.com/ns/aaa/1.1}aaa',
    '{http://cisco.com/ns/yang/Cisco-IOS-XR-aaa-lib-cfg}aaa',
    '{http://tail-f.com/ns/confd_dyncfg/1.0}confdConfig',
    '{http://tail-f.com/ns/mibs/SNMP-COMMUNITY-MIB/200308060000Z}SNMP-COMMUNITY-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-NOTIFICATION-MIB/200210140000Z}SNMP-NOTIFICATION-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-TARGET-MIB/200210140000Z}SNMP-TARGET-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-USER-BASED-SM-MIB/200210160000Z}SNMP-USER-BASED-SM-MIB',
    '{http://tail-f.com/ns/mibs/SNMP-VIEW-BASED-ACM-MIB/200210160000Z}SNMP-VIEW-BASED-ACM-MIB',
    '{http://tail-f.com/ns/mibs/SNMPv2-MIB/200210160000Z}SNMPv2-MIB',
    '{http://www.cisco.com/ns/yang/Cisco-IOS-XR-sysadmin-services}service',
    '{http://www.cisco.com/panini/calvados/sdr_mgr/sdr_sm}sdr-config',
    '{http://www.cisco.com/panini/calvados/sdr_mgr/sdr_sm}private-sdr',
    '{http://openconfig.net/yang/local-routing}local-routes',
    '{http://openconfig.net/yang/interfaces}interfaces'
]

# create a filtered before string
left_string = '<config>\n'
for item in etree.fromstring(running_config_before_xml).getchildren():
    if item.tag not in EXCL_TAGS:
        left_string = left_string + etree.tostring(item)
left_string = left_string + '</config>'

# create a filtered after string
right_string = '<config>\n'
for item in etree.fromstring(running_config_after_xml).getchildren():
    if item.tag not in EXCL_TAGS:
        right_string = right_string + etree.tostring(item)
right_string = right_string + '</config>'

# diff the filtered before & after
diffs = xtdiff.diff(etree.fromstring(left_string),
                    etree.fromstring(right_string))

# simple test
#
#left = """<root>
#</root>"""
#right = """<root>
#  <para>Lorem ipsum dolor sit amet</para>
#</root>"""
#xtdiff.diff(etree.fromstring(left), etree.fromstring(right))


In [ ]:
print(left_string)
print('')
print(right_string)
print('')
print(diffs)
print('')
left_root = etree.fromstring(left_string)
new_root = xtdiff.transform(left_root, diffs)
print(etree.tostring(new_root))

## Tidyup Sessions

In [ ]:
d.disconnect()

In [ ]:
m.close_session()